In [4]:
import spotpy 
import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from setup import SIRB_Setup, Results, ResultsDept
from sirb import SIRB
from misc import *
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

setup = SIRB_Setup().model_dept()
t0 = datetime.date(2010, 10, 24)
t1 = datetime.date(2012, 1, 1)
t2 = datetime.date(2016, 9, 30)
t3 = datetime.date(2016, 12, 1)
dept_name = ['Artibonite','Centre', 'Grande Anse', 'Nippes', 'Nord', 'Nord-Est', 'Nord-Ouest', 'Ouest', 'Sud' ,'Sud-Est']

cases = setup.cases[t0:t3]
rain = setup.rainfall.resample('W-SAT').sum()[t0:t3]

pop =cases.copy(deep=True)
pop['Ouest'] = 4029705
pop['Artibonite'] = 1727524
pop['Nord'] = 1067177
pop['Sud'] = 774976
pop['Centre'] = 746236
pop['Nord-Ouest'] = 728807
pop['Sud-Est'] = 632601
pop['Grande Anse'] = 468301
pop['Nippes'] = 342525
pop['Nord-Est'] = 393967

cases.cumsum()
susceptible = pop-cases.cumsum()
irn =  (cases.shift(1)+1)/(cases+1) #instantaneous reproduction number
irn = irn.replace([np.inf, -np.inf], np.nan)
max_irn = irn.max().max()
irn = irn.fillna(value=2*max_irn)

susceptibleoverpop = susceptible/pop

brn = irn * pop/susceptible

In [9]:


formula = 'CASES ~ RF + RFPR + np.log(CAPR) + np.log(SUSPOP)' # Complicated language: http://patsy.readthedocs.io/en/latest/formulas.html#operators Javier: no need for intersction stuff (would be if wash, 
#vaccine and wash and vaccine.)

dta = pd.concat([cases['Ouest'], 
                 cases['Ouest'].shift(1).fillna(1), 
                 rain['Ouest'], 
                 rain['Ouest'].shift(1).fillna(1), 
                 susceptibleoverpop['Ouest']],  
                keys = ['CASES', 
                        'CAPR', 
                        'RF',
                        'RFPR',
                        'SUSPOP'],axis = 1)[t0:t3]

dta = dta[t0:t3]

mod1 = smf.glm(formula=formula, data=dta, family=sm.families.Poisson()).fit()
# Javier: yes you can do that but not poisson ! + It is nb infected not incidence
#mod1 = smf.glm(formula=formula, data=dta, family=sm.families.Poisson()).fit()
#print(mod1.summary().as_latex())  # TODO: Has p-value and AIC
mod1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  CASES   No. Observations:                  318
Model:                            GLM   Df Residuals:                      313
Model Family:                 Poisson   Df Model:                            4
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -20417.
Date:                Sat, 14 Jul 2018   Deviance:                       38329.
Time:                        02:27:39   Pearson chi2:                 2.53e+05
No. Iterations:                     6   Covariance Type:             nonrobust
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          2.7741      0.024    117.736      0.000       2.728       2.820
RF                 0.0012   3.44e-05     35.723      0.000       0.001       0.001
RFPR               0.0017    3.2e-05     52.208      0.000       0.002       0.002
np.log(CAPR)       0.7027      0.003    276.491      0.000       0.698       0.708
np.log(SUSPOP)    14.7707      0.133    110.864      0.000      14.510      15.032
==================================================================================
"""

In [3]:
formula = 'CASES ~ RF + RFPR + np.log(CAPR) + np.log(SUSPOP)' 

cases_mean = cases.mean(axis=1)

rain_mean = rain.mean(axis=1)

sus_mean = susceptibleoverpop.median(axis=1)

dta = pd.concat([cases_mean, 
                 cases_mean.shift(1).fillna(1), 
                 rain_mean, 
                 rain_mean.shift(1).fillna(1), 
                 sus_mean],
                keys = ['CASES', 
                        'CAPR', 
                        'RF',
                        'RFPR',
                        'SUSPOP'],axis = 1)[t0:t3]

dta = dta[t0:t3]

mod1 = smf.glm(formula=formula, data=dta, family=sm.families.Poisson()).fit()
# Javier: yes you can do that but not poisson ! + It is nb infected not incidence
#mod1 = smf.glm(formula=formula, data=dta, family=sm.families.Poisson()).fit()
mod1.summary()

NameError: name 'smf' is not defined

In [44]:
formula = 'CASES ~ fnct(RF) + fnct(RFPR) + np.log(CAPR) + np.log(SUSPOP)' # Complicated language: http://patsy.readthedocs.io/en/latest/formulas.html#operators Javier: no need for intersction stuff (would be if wash, 
#vaccine and wash and vaccine.)

cases_mean = cases.mean(axis=1)

rain_mean = rain.mean(axis=1)

sus_mean = susceptibleoverpop.median(axis=1)

dta = pd.concat([cases_mean, 
                 cases_mean.shift(1).fillna(1), 
                 rain_mean, 
                 rain_mean.shift(1).fillna(1), 
                 sus_mean],
                keys = ['CASES', 
                        'CAPR', 
                        'RF',
                        'RFPR',
                        'SUSPOP'],axis = 1)[t0:t3]
dta = dta[t0:t3]
mm = [np.inf, 0]
for alpha in np.linspace(0,10, 1000):
    def fnct(data):
        return data**alpha
    mod1 = smf.glm(formula=formula, data=dta, family=sm.families.Poisson()).fit()
    if mod1.aic<mm[0]:
        mm[0] = mod1.aic
        mm[1] = alpha
    
    
# Javier: yes you can do that but not poisson ! + It is nb infected not incidence
#mod1 = smf.glm(formula=formula, data=dta, family=sm.families.Poisson()).fit()

In [33]:
import matplotlib as mpl
plt.style.use('seaborn-white')
params = {'legend.fontsize': 'small',
          'figure.figsize': (15, 5),
          'axes.labelsize': 'medium',
          'axes.titlesize':'medium',
          'xtick.labelsize':'medium',
          'ytick.labelsize':'medium',
          'font.serif':'Latin Modern Roman',
          'font.family': 'serif',
          'axes.labelweight': 'normal',
          'font.size': 22}
mpl.rcParams.update(params)
mpl.rc('font', family='serif')   
mpl.rc('font', serif='Latin Modern Roman')   
mpl.rcParams.update({'font.size': 12})     
mpl.rc('text', usetex=False)

def fnct(data, alpha):
    return data**alpha


fig, ax = plt.subplots(1, 1, figsize=(4,4), squeeze = True, dpi=400);



for i, alpha in enumerate([0, 1, 0.1, 10]):
    ax.plot(np.linspace(0,1,400), fnct(np.linspace(0,1,400), alpha),
            label = r'$\alpha$ = ' + str(alpha), marker = '', linewidth = 1, linestyle='-')
    
ax.set_xlabel(r'$\frac{J_t}{J_{max}}$')
ax.grid()
ax.set_title('Selective rainfall', fontsize=14)
ax.set_ylabel(r'$\frac{J_t^\alpha}{J_{max}^\alpha}$')

ax.legend(fontsize = )
fig.tight_layout()


IndentationError: expected an indented block (<ipython-input-33-0d4e994e04f2>, line 31)

In [47]:
mm

formula = 'CASES ~ fnct(RF) + fnct(RFPR) + np.log(CAPR) + np.log(SUSPOP)' # Complicated language: http://patsy.readthedocs.io/en/latest/formulas.html#operators Javier: no need for intersction stuff (would be if wash, 
#vaccine and wash and vaccine.)

cases_mean = cases.mean(axis=1)

rain_mean = rain.mean(axis=1)

sus_mean = susceptibleoverpop.median(axis=1)

dta = pd.concat([cases_mean, 
                 cases_mean.shift(1).fillna(1), 
                 rain_mean, 
                 rain_mean.shift(1).fillna(1), 
                 sus_mean],
                keys = ['CASES', 
                        'CAPR', 
                        'RF',
                        'RFPR',
                        'SUSPOP'],axis = 1)[t0:t3]
dta = dta[t0:t3]
mm = [np.inf, 0]
def fnct(data):
    return data**0.52
mod1 = smf.glm(formula=formula, data=dta, family=sm.families.Poisson()).fit()

    
print(mod1.summary().as_latex())


\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:} &      CASES       & \textbf{  No. Observations:  } &      318    \\
\textbf{Model:}         &       GLM        & \textbf{  Df Residuals:      } &      313    \\
\textbf{Model Family:}  &     Poisson      & \textbf{  Df Model:          } &        4    \\
\textbf{Link Function:} &       log        & \textbf{  Scale:             } &    1.0000   \\
\textbf{Method:}        &       IRLS       & \textbf{  Log-Likelihood:    } &   -6261.5   \\
\textbf{Date:}          & Sat, 14 Jul 2018 & \textbf{  Deviance:          } &    10411.   \\
\textbf{Time:}          &     03:07:08     & \textbf{  Pearson chi2:      } &  3.05e+04   \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                        & \textbf{coef} & \textbf{std err} & \textbf{z} & \textbf{P$>$$|$z$|$} & \textbf{[0.025} & \textbf{0.975]}  \\
\midrule
\textbf{Intercept}      &       3.8109  &        0.047     &    81.638  &         0.000        &        3.71